In [2]:
import scipy.io as scio
import pandas as pd
import os
import numpy as np
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.functional import interpolate
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torch.utils.data as Data
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from torch import einsum
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold, LeaveOneGroupOut
import copy
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, accuracy_score
#from sklearn import preprocessing
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from tqdm import tqdm, trange
from torcheeg.datasets.constants.emotion_recognition import format_region_channel_list
# from ConLoss import SupConLoss
import random
# import Module as md
import mne

from torcheeg.datasets.constants.emotion_recognition.deap import DEAP_GENERAL_REGION_LIST
from torcheeg.models import (EEGNet, FBCNet, TSCeption)
from torcheeg.models import DGCNN, ArjunViT, STNet, FBCNet, GRU, LSTM, LGGNet, MTCNN
# import MI as mi

G:\python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
eeg_data=np.load('data_epochs/eright_and_cwrong_data.npy')
dec_label=np.load('data_epochs/eright_and_cwrong_label_dec.npy')
emo_label=np.load('data_epochs/eright_and_cwrong_label_emo.npy')
eeg_data.shape,dec_label.shape,emo_label.shape

((2123, 67, 1251), (2123,), (2123,))

In [4]:
#将标签转化
# 创建字母到数字的映射字典
dec_judged_by_direction = {'1': 0, 
                           '2': 1, 
                           '3': 0,
                           '4': 1,
                           '5': 2}#0是左，1是右，2是没操作
dec_judged_by_correct = {'1': 0,
                         '2': 0,
                         '3': 1,
                         '4': 1,
                         '5': 1}#0是对，1是错
emo_judged_by_movie = {'01': 0, 
                       '02': 1, 
                       '11': 2,
                       '12': 3,
                       '21': 4,
                       '22': 5,
                       '31': 6,
                       '32': 7}
emo_judged_by_emotion = {'01': 0, 
                       '02': 0, 
                       '11': 1,
                       '12': 1,
                       '21': 2,
                       '22': 2,
                       '31': 3,
                       '32': 3}

In [5]:
# 
#使用映射字典进行转换
dec_convert_label = np.array([dec_judged_by_correct [letter] for letter in dec_label])
emo_convert_label = np.array([emo_judged_by_emotion [letter] for letter in emo_label])

In [6]:
def set_seed_everywhere(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

In [7]:
#打乱数据
set_seed_everywhere(3407)
index=np.random.permutation(eeg_data.shape[0])
print(index)
eeg_data=eeg_data[index]
delete_index=[32,42,64,65,66]
eeg_data=np.delete(eeg_data,delete_index,axis=1)
dec_label=dec_convert_label[index]
emo_label=emo_convert_label[index]

[1533 2000  113 ... 1140  614  580]


In [8]:
#训练数据
eegdata_train=eeg_data[:int(np.floor(len(eeg_data)*0.8))]
declabel_train=dec_label[:int(np.floor(len(eeg_data)*0.8))]
emolabel_train=emo_label[:int(np.floor(len(eeg_data)*0.8))]

#测试数据
eegdata_test=eeg_data[int(np.floor(len(eeg_data)*0.8)):]
emolabel_test=emo_label[int(np.floor(len(eeg_data)*0.8)):]
declabel_test=dec_label[int(np.floor(len(eeg_data)*0.8)):]

eegdata_test.shape,emolabel_test.shape,declabel_test.shape

((425, 62, 1251), (425,), (425,))

In [9]:
train_data=torch.from_numpy(eegdata_train.reshape(-1,1,62,1251)).float()
train_dec_label=torch.from_numpy(declabel_train).float()
train_emo_label=torch.from_numpy(emolabel_train).float()


test_data=torch.from_numpy(eegdata_test.reshape(-1,1,62,1251)).float()
test_dec_label=torch.from_numpy(declabel_test).float()
test_emo_label=torch.from_numpy(emolabel_test).float()
train_data.shape,train_dec_label.shape,train_emo_label.shape,test_data.shape,test_dec_label.shape,test_emo_label.shape

(torch.Size([1698, 1, 62, 1251]),
 torch.Size([1698]),
 torch.Size([1698]),
 torch.Size([425, 1, 62, 1251]),
 torch.Size([425]),
 torch.Size([425]))

In [10]:
#去除操作部分数据，只保留操作前的数据
train_data=train_data[:,:,:,:875]
test_data=test_data[:,:,:,:875]
train_data.shape,train_dec_label.shape,train_emo_label.shape,test_data.shape,test_dec_label.shape,test_emo_label.shape

(torch.Size([1698, 1, 62, 875]),
 torch.Size([1698]),
 torch.Size([1698]),
 torch.Size([425, 1, 62, 875]),
 torch.Size([425]),
 torch.Size([425]))

In [11]:
'''
归一化处理
'''
def scale(X, dim):
    X -= np.mean(X, axis=dim, keepdims=True)
    return X / np.std(X, axis=dim, keepdims=True)

train_data = scale(train_data.numpy(), dim=3)
test_data = scale(test_data.numpy(), dim=3)


In [12]:
train_data=torch.from_numpy(train_data)
test_data=torch.from_numpy(test_data)

In [13]:

np.save(f'data_training/eright_and_cwrong_train_data.npy',train_data)
np.save(f'data_training/eright_and_cwrong_train_dec_label.npy',train_dec_label)
np.save(f'data_training/eright_and_cwrong_train_emo_label.npy',train_emo_label)
np.save(f'data_training/eright_and_cwrong_test_data.npy',test_data)
np.save(f'data_training/eright_and_cwrong_test_dec_label.npy',test_dec_label)
np.save(f'data_training/eright_and_cwrong_test_emo_label.npy',test_emo_label)